In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
#------------------------------------------------------------------------------- 
# Optional generic preliminaries:
# graphics.off() # This closes all of R's graphics windows.
# rm(list=ls())  # Careful! This clears all of R's memory!
#------------------------------------------------------------------------------- 
#Load The data file 
#.............................................................................
myDataFrame = read.csv( file="HairEyeColor.csv" )
# Alter count by a multiplier, for demo purposes:
countMult = 1
myDataFrame$Count = round( myDataFrame$Count * countMult )
fileNameRoot = paste0("HairEyeColor-",countMult,"-") 
yName="Count" 
x1Name="Eye" 
x2Name="Hair"  
x1contrasts = list( 
  list( c("Green") , c("Hazel") , compVal=0.0 , ROPE=c(-0.1,0.1) ) ,
  list( c("Blue")  , c("Green") , compVal=0.0 , ROPE=c(-0.1,0.1) )  
)
x2contrasts = list( 
  list( c("Black") , c("Blond") , compVal=0.0 , ROPE=c(-0.1,0.1) ) ,
  list( c("Brown") , c("Red")   , compVal=0.0 , ROPE=c(-0.1,0.1) )  
)
x1x2contrasts = list( 
  list( list( c("Blue") , c("Brown") ) ,
        list( c("Black") , c("Blond") ) ,
        compVal=0.0 , ROPE=c(-0.1,0.1) ) 
) 
numSavedSteps = 12000
thinSteps = 10

graphFileType = "eps" 
#------------------------------------------------------------------------------- 
# Load the relevant model into R's working memory:
source("Jags-Ycount-Xnom2fac-MpoissonExp.R")
#------------------------------------------------------------------------------- 



*********************************************************************
Kruschke, J. K. (2015). Doing Bayesian Data Analysis, Second Edition:
A Tutorial with R, JAGS, and Stan. Academic Press / Elsevier.
*********************************************************************



Loading required package: coda
Linked to JAGS 4.3.0
Loaded modules: basemod,bugs

Attaching package: ‘runjags’

The following object is masked from ‘package:tidyr’:

    extract



In [3]:
myDataFrame

Hair,Eye,Count
Black,Blue,20
Black,Brown,68
Black,Green,5
Black,Hazel,15
Blond,Blue,94
Blond,Brown,7
Blond,Green,16
Blond,Hazel,10
Brown,Blue,84
Brown,Brown,119


In [49]:
states <- c("s1", "s2", "s3")
actions <- c("a1", "a2", "a3")

df <- data.frame(states, actions)

In [50]:
df <- df %>% expand(states, actions)

In [57]:
rewards <- round(c(2, 1, 4, 1, 5, 2, 1, 1, 4))

In [58]:
df["rewards"] <- rewards 

In [59]:
df

states,actions,rewards
s1,a1,2
s1,a2,1
s1,a3,4
s2,a1,1
s2,a2,5
s2,a3,2
s3,a1,1
s3,a2,1
s3,a3,4


In [9]:
yName="reward" 
x1Name="state" 
x2Name="action"

In [61]:
myDataFrame = df

In [10]:
myDataFrame = read.csv( file="states.csv" )

In [11]:
# Generate the MCMC chain:
mcmcCoda = genMCMC( datFrm=myDataFrame , 
                    yName=yName , x1Name=x1Name , x2Name=x2Name ,
                    numSavedSteps=numSavedSteps , thinSteps=thinSteps )

    shape      rate 
1.6403882 0.1425972 
Calling 4 simulations using the parallel method...
Following the progress of chain 1 (the program will wait for all chains
to finish before continuing):
Welcome to JAGS 4.3.0 on Mon Jan 29 10:01:19 2018
JAGS is free software and comes with ABSOLUTELY NO WARRANTY
Loading module: basemod: ok
Loading module: bugs: ok
. . Reading data file data.txt
. Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 12
   Unobserved stochastic nodes: 23
   Total graph size: 178

ySum

. Reading parameter file inits1.txt
. Initializing model
. Adapting 1000
-------------------------------------------------| 1000
++++++++++++++++++++++++++++++++++++++++++++++++++ 100%
Adaptation successful
. Updating 2000
-------------------------------------------------| 2000
************************************************** 100%
. . . . . . . . . . . . Updating 30000
----------------------------------------

In [7]:

#------------------------------------------------------------------------------- 
# Display diagnostics of chain, for specified parameters:
parameterNames = varnames(mcmcCoda) 
show( parameterNames ) # show all parameter names, for reference
for ( parName in c("b0","b1[1]","b2[1]","b1b2[1,1]","ppx1x2p[1,1]",
                   "a1SD","a1a2SD") ) {
  diagMCMC( codaObject=mcmcCoda , parName=parName , 
            saveName=fileNameRoot , saveType=graphFileType )
}


 [1] "ppx1x2p[1,1]" "ppx1x2p[2,1]" "ppx1x2p[3,1]" "ppx1x2p[1,2]" "ppx1x2p[2,2]"
 [6] "ppx1x2p[3,2]" "ppx1x2p[1,3]" "ppx1x2p[2,3]" "ppx1x2p[3,3]" "ppx1p[1]"    
[11] "ppx1p[2]"     "ppx1p[3]"     "ppx2p[1]"     "ppx2p[2]"     "ppx2p[3]"    
[16] "b0"           "b1[1]"        "b1[2]"        "b1[3]"        "b2[1]"       
[21] "b2[2]"        "b2[3]"        "b1b2[1,1]"    "b1b2[2,1]"    "b1b2[3,1]"   
[26] "b1b2[1,2]"    "b1b2[2,2]"    "b1b2[3,2]"    "b1b2[1,3]"    "b1b2[2,3]"   
[31] "b1b2[3,3]"    "m[1,1]"       "m[2,1]"       "m[3,1]"       "m[1,2]"      
[36] "m[2,2]"       "m[3,2]"       "m[1,3]"       "m[2,3]"       "m[3,3]"      
[41] "a1SD"         "a2SD"         "a1a2SD"      


In [10]:
??plotMCMC

In [12]:
#------------------------------------------------------------------------------- 
# Get summary statistics of chain:
summaryInfo = smryMCMC( mcmcCoda , 
                        datFrm=myDataFrame , x1Name=x1Name , x2Name=x2Name ,
#                         x1contrasts=x1contrasts , 
#                         x2contrasts=x2contrasts , 
#                         x1x2contrasts=x1x2contrasts ,
                        saveName=fileNameRoot )
show(summaryInfo)
# Display posterior information:
plotMCMC( mcmcCoda , 
          datFrm=myDataFrame , yName=yName , x1Name=x1Name , x2Name=x2Name ,
#           x1contrasts=x1contrasts , 
#           x2contrasts=x2contrasts , 
#           x1x2contrasts=x1x2contrasts ,
          saveName=fileNameRoot , saveType=graphFileType )

                           Mean       Median          Mode     ESS HDImass
ppx1x2p[1,1] s1 a1  0.004395726  0.003437910  0.0014167924  3348.4    0.95
ppx1x2p[2,1] s2 a1  0.322170992  0.321970500  0.3228610298 12000.0    0.95
ppx1x2p[3,1] s3 a1  0.322401483  0.322310000  0.3241502795 12000.0    0.95
ppx1x2p[4,1] s4 a1  0.003977002  0.003026310  0.0012743292  8444.7    0.95
ppx1x2p[1,2] s1 a2  0.321800415  0.321689500  0.3220192264 12000.0    0.95
ppx1x2p[2,2] s2 a2  0.003934433  0.003008505  0.0012060263  5268.0    0.95
ppx1x2p[3,2] s3 a2  0.003947179  0.003005865  0.0010890440  6045.9    0.95
ppx1x2p[4,2] s4 a2  0.003608813  0.002684295  0.0009333701  7451.3    0.95
ppx1x2p[1,3] s1 a3  0.003567374  0.002647120  0.0009297260  9220.5    0.95
ppx1x2p[2,3] s2 a3  0.003491011  0.002556975  0.0010385426  7264.7    0.95
ppx1x2p[3,3] s3 a3  0.003525733  0.002625690  0.0008938744 10751.3    0.95
ppx1x2p[4,3] s4 a3  0.003179835  0.002239160  0.0007621583 12000.0    0.95
ppx1p[1]            0.329